<a href="https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/main/solver_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/master/solver_experiments.ipynb)

Notebook experimenting how to use DOcplex and IBM Quantum.

# 0. Install libraries & clone repo

In [8]:
import sys

if hasattr(sys, 'real_prefix'):
    #we are in a virtual env.
    !pip install cplex && pip install docplex
else:
    !pip install --user cplex && pip install --user docplex

print()
!pip show cplex
print()
!pip show docplex


Name: cplex
Version: 20.1.0.1
Summary: A Python interface to the CPLEX Callable Library, Community Edition.
Home-page: https://www.ibm.com/analytics/data-science/prescriptive-analytics/cplex-optimizer/
Author: IBM
Author-email: UNKNOWN
License: Proprietary
Location: /root/.local/lib/python3.7/site-packages
Requires: 
Required-by: 
Name: docplex
Version: 2.21.207
Summary: The IBM Decision Optimization CPLEX Modeling for Python
Home-page: https://onboarding-oaas.docloud.ibmcloud.com/software/analytics/docloud/
Author: The IBM Decision Optimization on Cloud team
Author-email: dofeedback@wwpdl.vnet.ibm.com
License: Apache 2.0
Location: /root/.local/lib/python3.7/site-packages
Requires: requests, six
Required-by: 


Possibly need to restart runtime here.

In [2]:
# clone repo
!git clone https://github.com/VGGatGitHub/Summer2021.git

Cloning into 'Summer2021'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 37 (delta 5), reused 7 (delta 2), pack-reused 0
Unpacking objects: 100% (37/37), done.


# 1. Load Data

In [1]:
#next run the model.py in the folder introScenario
%pwd
%cd ./Summer2021/introScenario/
#%ls


/content/Summer2021/introScenario


In [2]:
import pandas as pd

# supply + costs
supply_file = './plants.csv'

# demand
demand_file = './customerDemand.csv'

# for now load example data (plants.csv and customerDemand.csv under introScenario folder)
s_df = pd.read_csv(supply_file)
d_df = pd.read_csv(demand_file)

d_df[u'Product']


0    handSanitizer
1             mask
Name: Product, dtype: object

In [9]:
d_data=open(demand_file, "r").read()
s_data=open(supply_file, "r").read()
inputs={'customerDemand':d_data,'plants':s_data}
inputs

{'customerDemand': 'Product,Demand\nhandSanitizer,100\nmask,120\n',
 'plants': 'Plants,Cost,Capacity,Product\n1,3,40,mask\n2,2,30,mask\n3,1,30,handSanitizer\n4,3,100,handSanitizer\n5,2,60,mask\n6,1,45,mask\n'}

In [3]:
inputs={'customerDemand':d_df,'plants':s_df}


In [5]:
global output_lock
output_lock={}
exec(open("./model.py").read())

* building wado model
model building done in 0.07221746444702148 secs
* running wado model
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 20 rows and 12 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.01 ticks)
Solve status: JobSolveStatus.OPTIMAL_SOLUTION
* model docplex_model1 solved with objective = -105680.000
*  KPI: total plants allocations                  = 112640.000
*  KPI: total Cost of plants over all all

AttributeError: ignored

In [6]:
msol

docplex.mp.solution.SolveSolution(obj=-105680,values={x2:15,x3:30,x4:70,..

# 2. Solve the DO problem via DOcplex

In [4]:
#If you get an error you may have to restart the runtime
import cplex
import docplex.mp
from docplex.mp.model import Model
# TODO

# 3. Solve the DO problem using IBM Quantum

In [ ]:
# TODO (after steps 1 and 2)